<a href="https://colab.research.google.com/github/IgorNascAlves/Validacao-de-chaves-aleatorias-FIPS-140-1/blob/master/Valida%C3%A7%C3%A3o_de_chaves_aleat%C3%B3rias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install BitVector

In [2]:
import pandas as pd
from collections import Counter
from BitVector import BitVector

In [3]:
URI = './chaves.txt'
data = pd.read_csv(URI, sep="'", header=None)
data = data.drop(columns=[0, 2])
data = pd.DataFrame([f"{BitVector(hexstring = item)}" for item in data[1]])

# The Monobit Test

In [4]:
#9654 < X < 10346

In [5]:
contador = []
for chave in data[0].values:
  c = Counter(chave)
  contador.append(c['1'])

In [6]:
chaves_fail_monobit = []
i = 0
ind = 0
for item in contador:
  if item > 9654 and item < 10346:
    i = i + 1
  else:
    chaves_fail_monobit.append(ind)
  ind = ind + 1
print(f"No The Monobit Test do total de {len(contador)} chaves, {i} chaves passaram")

No The Monobit Test do total de 20 chaves, 10 chaves passaram


# The Poker Test

In [7]:
#Divide the 20,000 bit stream into 5,000 contiguous 4 bit segments.
contador_por_chave = []
contador = []
nibble = ''
i = 0
for chave in data[0].values:
  for valor in chave:
    if i == 4:
      contador.append(nibble)
      i = 0
      nibble = ''
    nibble = nibble + valor
    i = i + 1
  contador_por_chave.append(contador)
  contador = []
contador_por_chave = pd.DataFrame(contador_por_chave)

In [8]:
#Count and store the number of occurrences of each of the 16 possible 4 bit values.
nibbles = ['0000','0001','0010','0011',
           '0100','0101','0110','0111',
           '1000','1001','1010','1011',
           '1100','1101','1110','1111',
           ]
nibble_por_chave = {}          
total_nibble_por_chave = []
for chave in contador_por_chave.values:           
  for nibble in nibbles:
    c = Counter(chave)
    nibble_por_chave[nibble] = c[nibble]
  total_nibble_por_chave.append(nibble_por_chave)
  nibble_por_chave = {}
total_nibble_por_chave = pd.DataFrame(total_nibble_por_chave)

In [9]:
#Denote f(i) as the number of each 4 bit value i where 0 s is 15.
soma_chaves = []
acumulado = 0
x = 0
#chave = total_nibble_por_chave[:1]
for chave in total_nibble_por_chave.values:
  for valor in chave:
    acumulado = acumulado + valor * valor
  x = (16/5000) * acumulado - 5000
  soma_chaves.append(x)
  x = 0
  acumulado = 0
soma_chaves = pd.DataFrame(soma_chaves)

In [10]:
#The test is passed if 1.03 < X < 57.4.
chaves_fail_poker = []
ind = 0
i = 0
for chave in soma_chaves.values:
  if chave > 1.03 and chave < 57.4:
    i = i + 1
  else:
    chaves_fail_poker.append(ind)
  ind = ind + 1
print(f"No The Poker Test do total de {len(soma_chaves)} chaves, {i} chaves passaram")    

No The Poker Test do total de 20 chaves, 10 chaves passaram


# The Run Test

In [11]:
contador_label = {
    '1': 0,
    '2': 0,
    '3': 0,
    '4': 0,
    '5': 0,
    '6': 0,
}
contador_por_chave = []
salvo = ''
i = 0
j = 0
for ind in range(0,20,1):
  chave = data[0].values[ind]
  for item in chave:
    if salvo == '':
      salvo = item
      i = i + 1
    else:  
      if item == salvo:
        i = i + 1
      else:
        if i >= 6:
          contador_label['6'] = contador_label['6'] + 1
        if i == 5:
          contador_label['5'] = contador_label['5'] + 1
        if i == 4:
          contador_label['4'] = contador_label['4'] + 1
        if i == 3:
          contador_label['3'] = contador_label['3'] + 1
        if i == 2:
          contador_label['2'] = contador_label['2'] + 1
        if i == 1:
          contador_label['1'] = contador_label['1'] + 1
        i = 0
        salvo = item
  contador_por_chave.append(contador_label)
  contador_label = {
    '1': 0,
    '2': 0,
    '3': 0,
    '4': 0,
    '5': 0,
    '6': 0,
  }
  salvo = ''
contador_por_chave = pd.DataFrame(contador_por_chave)

In [12]:
#1 2267 - 2733
#2 1079 - 1421
#3  502 - 748
#4  223 - 402
#5   90 - 223 
#6+  90 - 223

In [13]:
chaves_fail_run = []
ind = 0
i = 0
for contador in contador_por_chave.values:
  if contador[5] < 90 or contador[5] > 223:
    chaves_fail_run.append(ind)
  elif contador[4] < 90 or contador[4] > 223:
    chaves_fail_run.append(ind)
  elif contador[3] < 223 or contador[3] > 402:
    chaves_fail_run.append(ind)
  elif contador[2] < 502 or contador[2] > 748:
    chaves_fail_run.append(ind)
  elif contador[1] < 1079 or contador[1] > 1421:
    chaves_fail_run.append(ind)
  elif contador[0] < 2267 or contador[0] > 2733:
    chaves_fail_run.append(ind)
  ind = ind + 1
print(f"No The Run Test, do total de {len(contador_por_chave)} chaves, {len(contador_por_chave) - len(chaves_fail_run)} chaves passaram")

No The Run Test, do total de 20 chaves, 13 chaves passaram


# The Long Run Test

In [14]:
#long run = len(1) or len(0) >= 34
#passed id no long run

In [16]:
contador_por_chave = []
contador = []
salvo = ''
i = 0
j = 0
for ind in range(0,20,1):
  chave = data[0].values[ind]
  for item in chave:
    if salvo == '':
      salvo = item
    else:  
      if item == salvo:
        i = i + 1
      else:
        contador.append(i)
        i = 0
        salvo = item
  contador_por_chave.append(contador)
  contador = []
  salvo = ''
contador_por_chave = pd.DataFrame(contador_por_chave)

In [17]:
chaves_fail_long = []
ind = 0
i = 0
for contador in contador_por_chave.values:
  if len(contador[contador > 33]) == 0:
    i = i + 1
  else:
    chaves_fail_long.append(ind)
  ind = ind + 1
print(f"No The Long Run Test, do total de {len(contador_por_chave)} chaves, {i} chaves passaram")

No The Long Run Test, do total de 20 chaves, 20 chaves passaram


# Testes

In [18]:
print("Monobit falharam:", len(chaves_fail_monobit), chaves_fail_monobit)
print("Poker falharam:", len(chaves_fail_poker), chaves_fail_poker)
print("Run falharam:", len(chaves_fail_run), chaves_fail_run)
print("Long Run falharam:", len(chaves_fail_long), chaves_fail_long)

Monobit falharam: 10 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Poker falharam: 10 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Run falharam: 7 [2, 3, 4, 8, 9, 10, 11]
Long Run falharam: 0 []
